In [3]:
#Load the required modules
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Input, BatchNormalization
from keras.constraints import maxnorm
from keras.models import Model
from keras.optimizers import *
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
import tensorflow as tf

In [4]:
#Load the dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170508288/170498071 [==============================] - 2s 0us/step


In [5]:
#Normalize the inputs for better performance from the rang eof 0 to 1
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [6]:
#One hot encoding as we are going to use categorical entropy as loss function
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

**Sequential Model**

In [7]:
#create the sequential model and add the required Conv2D,Dropout,MaxPooling2D,Dense and Flatten layers
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(32,32, 3), padding='same', activation='relu'))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

In [12]:
#Compile the model using categorical cross entropy and optimizer as adam and metrics as accuracy 
epochs = 5
lrate = 0.001
adam = tf.keras.optimizers.Adam(learning_rate=lrate)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [13]:
#print model summary
print(model.summary)

<bound method Model.summary of <keras.engine.sequential.Sequential object at 0x7f299568e7d0>>


In [15]:
#fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=128, verbose=1)

Epoch 1/5
391/391 [==============================] - 389s 993ms/step - loss: 1.4432 - accuracy: 0.4742 - val_loss: 1.2472 - val_accuracy: 0.5518
Epoch 2/5
391/391 [==============================] - 391s 999ms/step - loss: 1.1529 - accuracy: 0.5886 - val_loss: 1.0283 - val_accuracy: 0.6398
Epoch 3/5
391/391 [==============================] - 390s 997ms/step - loss: 0.9705 - accuracy: 0.6545 - val_loss: 0.8748 - val_accuracy: 0.6941
Epoch 4/5
391/391 [==============================] - 391s 1s/step - loss: 0.8391 - accuracy: 0.7031 - val_loss: 0.8038 - val_accuracy: 0.7195
Epoch 5/5
391/391 [==============================] - 391s 1s/step - loss: 0.7474 - accuracy: 0.7386 - val_loss: 0.7177 - val_accuracy: 0.7495


In [16]:
#Print the accuracy by evaluating the model 
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 74.95%


**Accuracy of sequential model is 74%**

**Functional API**

In [17]:
#create the Input layer and required Conv2D,Dropout,MaxPooling2D,Dense and Flatten layers
input = Input(shape = X_train[0].shape)
conv1 = Conv2D(32, (3,3), padding="same", activation="relu")(input)
dropout1 = Dropout(0.2)(conv1)
conv2 = Conv2D(32, (3,3), activation="relu", padding="same",)(dropout1)
maxpool1 = MaxPooling2D(pool_size=(2,2))(conv2)
conv3 = Conv2D(64, (3,3), padding="same", activation="relu")(maxpool1)
dropout2 = Dropout(0.2)(conv3)
conv4 = Conv2D(64, (3,3), padding="same", activation="relu")(dropout2)
maxpool2 = MaxPooling2D(pool_size=(2,2))(conv4)
conv5 = Conv2D(128, (3,3), padding="same", activation="relu")(maxpool2)
dropout3 = Dropout(0.2)(conv5)
conv6 = Conv2D(128, (3,3), padding="same", activation="relu")(dropout3)
maxpool3 = MaxPooling2D(pool_size=(2,2))(conv6)
flatten1 = Flatten()(maxpool3)
dense1 = Dense(1024, activation="relu")(flatten1)
flatten2 = Dropout(0.2)(dense1)
dense2 = Dense(512, activation="relu")(flatten2)
dropout4 = Dropout(0.2)(dense2)
output = Dense(num_classes, activation="softmax")(dropout4)
model = Model(input, output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 dropout_6 (Dropout)         (None, 32, 32, 32)        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 16, 16, 64)        18496     
                                                             

In [20]:
#compile the model
epochs = 5
lrate = 0.001
adam = tf.keras.optimizers.Adam(learning_rate=lrate)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [21]:
#print the model summary
print(model.summary)

<bound method Model.summary of <keras.engine.functional.Functional object at 0x7f2987c6f590>>


In [22]:
#fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=128, verbose=1)

Epoch 1/5
391/391 [==============================] - 382s 974ms/step - loss: 1.6771 - accuracy: 0.3766 - val_loss: 1.3454 - val_accuracy: 0.5241
Epoch 2/5
391/391 [==============================] - 387s 991ms/step - loss: 1.1859 - accuracy: 0.5740 - val_loss: 1.0134 - val_accuracy: 0.6426
Epoch 3/5
391/391 [==============================] - 390s 997ms/step - loss: 0.9510 - accuracy: 0.6607 - val_loss: 0.9529 - val_accuracy: 0.6663
Epoch 4/5
391/391 [==============================] - 387s 990ms/step - loss: 0.8138 - accuracy: 0.7153 - val_loss: 0.8053 - val_accuracy: 0.7153
Epoch 5/5
391/391 [==============================] - 387s 990ms/step - loss: 0.7028 - accuracy: 0.7529 - val_loss: 0.7815 - val_accuracy: 0.7307


In [23]:
#print the accuracy of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 73.07%


**Accuracy of functional model is 73%**